# 영화 평점 크롤링

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('d:/2nd_project/Data/movie_data/[KOBIS] 박스오피스_줄거리_감정키워드(2003.01~2023.07).xlsx', engine='openpyxl')
df

,영화명,개봉일,누적관객수,등급,장르,대표국적,국적,제작사,배급사,감독,배우,줄거리,키워드
0,#살아있다,2020-06-24,1903992,15세이상관람가,드라마,한국,한국,"영화사 집,(주)퍼스펙티브픽쳐스",롯데컬처웍스(주)롯데엔터테인먼트,조일형,"유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...",NaN,NaN
1,#아이엠히어,2021-01-14,16404,12세이상관람가,"멜로/로맨스,코미디",프랑스,프랑스,NaN,(주)넥스트엔터테인먼트월드(NEW),에릭 라티고,"알랭 샤바,배두나,이명자,선종남,김자금,강현중,정윤하,송경의,엄지만,남승화,이지민,...",NaN,NaN
2,#위왓치유,2021-06-03,1908,청소년관람불가,다큐멘터리,기타,기타,NaN,찬란,"바르보라 차르포바,비트 클루삭",NaN,NaN,NaN
3,...ing,2003-11-28,34308,12세관람가,"드라마,멜로/로맨스",한국,한국,드림맥스(주),(주)튜브엔터테인먼트,이언희,"임수정,김래원,이미숙,김지영,김현수,박혜연,장미,이성경,박소연,김순애,최덕문,김인문...",홀엄마가 뽀뽀하자고 달려들면 '욕구 불만이야. 애인을 구해봐.'라고 얘기하는 여고생...,NaN
4,0.0MHz,2019-05-29,137294,15세이상관람가,공포(호러),한국,한국,(주)제이엠컬쳐스,(주)스마일이엔티,유선동,"정은지,이성열,최윤영,신주환,정원창,박명신,남관효,박대희,노혜영,서진석,김대현,신승...","에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,항구의 니쿠코짱!,2023-04-27,10539,전체관람가,애니메이션,일본,일본,NaN,트윈플러스파트너스(주),와타나베 아유무,"오타케 시노부,하나에 나츠키",﻿“보통날이 제일 좋은 거야!” 평범한 인생도 ‘갓생’으로 만드는 무한 긍정 에너지...,NaN
18416,해시태그 시그네,2023-01-11,4411,15세이상관람가,"멜로/로맨스,코미디",노르웨이,노르웨이,NaN,판씨네마(주),NaN,NaN,카페 바리스타로 따분한 인생을 살던 '시그네'에겐 행위 예술가로 매거진 표지를 장식...,NaN
18417,헌티드 맨션,2023-07-26,11075,12세이상관람가,공포(호러),미국,미국,NaN,월트디즈니컴퍼니코리아 유한책임회사,저스틴 시미엔,"티파니 해디쉬,오웬 윌슨,대니 드비토,로자리오 도슨",디즈니 &amp; &lt;캐리비안의 해적&gt; 제작진의 오싹한 초대장! 뉴올리언스...,NaN
18418,호쿠사이,2023-07-16,339,12세이상관람가,드라마,일본,일본,NaN,(주)미디어캐슬,하시모토 하지메,"야기라 유야,타나카 민,타마키 히로시,타키모토 미오리,츠다 칸지,아오키 무네타카",호화찬란한 상인문화로 채색된 에도 거리 한쪽 구석에 싹트지 않는 화가가 한 명이 있...,NaN


In [3]:
movie_lists = []
movie_lists = df['영화명']
movie_lists

0            #살아있다
1           #아이엠히어
2            #위왓치유
3           ...ing
4           0.0MHz
           ...    
18415    항구의 니쿠코짱!
18416     해시태그 시그네
18417       헌티드 맨션
18418         호쿠사이
18419          흐르다
Name: 영화명, Length: 18420, dtype: object

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert

In [10]:
# # 셀레니움으로 url 주소 크롬 열기
# driver = webdriver.Chrome()
# url = 'https://m.search.naver.com/search.naver'

# driver.get(url)
# time.sleep(1)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 116.0.5845.96 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0058A813+48355]
	(No symbol) [0x0051C4B1]
	(No symbol) [0x00425358]
	(No symbol) [0x004461AC]
	(No symbol) [0x00441EF3]
	(No symbol) [0x00440579]
	(No symbol) [0x00470C55]
	(No symbol) [0x0047093C]
	(No symbol) [0x0046A536]
	(No symbol) [0x004482DC]
	(No symbol) [0x004493DD]
	GetHandleVerifier [0x007EAABD+2539405]
	GetHandleVerifier [0x0082A78F+2800735]
	GetHandleVerifier [0x0082456C+2775612]
	GetHandleVerifier [0x006151E0+616112]
	(No symbol) [0x00525F8C]
	(No symbol) [0x00522328]
	(No symbol) [0x0052240B]
	(No symbol) [0x00514FF7]
	BaseThreadInitThunk [0x75E900C9+25]
	RtlGetAppContainerNamedObjectPath [0x77617B1E+286]
	RtlGetAppContainerNamedObjectPath [0x77617AEE+238]


In [27]:
# # 검색 영화명 
# question = f"영화+{movie_lists[0]}+평점"

# # 검색창 클릭
# driver.find_element(By.XPATH,'//*[@id="nx_query"]').click()
# time.sleep(0.5)
# driver.find_element(By.XPATH,'//*[@id="nx_query"]').send_keys(Keys.CONTROL + "A")
# time.sleep(0.5)
# driver.find_element(By.XPATH,'//*[@id="nx_query"]').send_keys(question)
# time.sleep(0.5)
# driver.find_element(By.XPATH,'//*[@id="nx_query"]').send_keys(Keys.ENTER)

In [31]:
# req = requests.get(url)
# html = req.text
# soup = BeautifulSoup(html, 'html.parser')
# soup

<!DOCTYPE html>
 <html lang="ko"> <head> <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/> <meta content="text/javascript" http-equiv="Content-Script-Type"/> <meta content="always" name="referrer"/> <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0, minimum-scale=1.0, user-scalable=yes" name="viewport"/> <meta content="telephone=no,address=no,email=no" name="format-detection"/> <title> :: 네이버 통합검색검색</title> <link href="https://ssl.pstatic.net/sstatic/search/mobile/css/ws_210527.css" rel="stylesheet" type="text/css"/> <link href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118.ico" rel="shortcut icon"/> </head><script> if (!String.prototype.trim) { String.prototype.trim = function () { return this.replace(/^[\s\uFEFF\xA0]+|[\s\uFEFF\xA0]+$/g, ''); }; } if (!Array.prototype.indexOf) { Array.prototype.indexOf = function(searchElement, fromIndex) { var k; if (this == null) { throw new TypeError('"this" is null or not defined'); } var o

In [33]:
# soup.select('.area_intro_info')

[]

In [ ]:
data = pd.DataFrame(columns = ['영화명', '관람객 평점', '남자 평점', '여자 평점',
                               '10대 평점', '20대 평점', '30대 평점', '40대 평점', '50대 평점',
                               '리뷰 평점', '리뷰', '포스터'])

base_url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query='
driver = webdriver.Chrome()
driver.get(base_url)

ec = 0

for movie_list in movie_lists:
    try:
        sup_url = f'영화+{movie_list}+평점'
        url = base_url + sup_url
        driver.get(url)
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'html.parser')

        # 영화명
        mname = movie_list
        print(mname)

        # 기본정보 클릭
        time.sleep(0.3)
        driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[1]/div[3]/div/div/ul/li[2]/a').click()

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # 포스터
        movie_img = soup.select('.detail_info > a')[0].select('img')[0].get_text()

        # # 개봉일
        # date = soup.select('.detail_info > dl')[0].select('div')[0].get_text().strip().split(' ')[1]
        # print(date)
        
        # # 러닝타임
        # runtime = soup.select('.detail_info > dl')[0].select('div')[4].get_text().strip().split(' ')[1]
        
        # # 평점 클릭
        time.sleep(0.3)
        driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[1]/div[3]/div/div/ul/li[4]/a').click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # 전체 평점
        tot_score = float(soup.select('.area_star_number')[0].get_text())
        # 남자 평점
        man_score = float(soup.select('.area_star_number')[1].get_text())
        # 여자 평점
        girl_score = float(soup.select('.area_star_number')[2].get_text())

        # 10대 만족도
        age10_score = float(soup.select('.area_graph_age > li')[0].select('.this_text_num')[0].get_text())
        # 20대 만족도
        age20_score = float(soup.select('.area_graph_age > li')[1].select('.this_text_num')[0].get_text())
        # 30대 만족도
        age30_score = float(soup.select('.area_graph_age > li')[2].select('.this_text_num')[0].get_text())
        # 40대 만족도
        age40_score = float(soup.select('.area_graph_age > li')[3].select('.this_text_num')[0].get_text())
        # 50대 만족도
        age50_score = float(soup.select('.area_graph_age > li')[4].select('.this_text_num')[0].get_text())

        # 공감순 리뷰 페이지 내리기
        time.sleep(0.3)
        driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[2]/div/div[2]/div[2]/div/div[1]/ul/li[2]/a').click()

        time.sleep(0.3)
        driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[2]/div/div[2]/div[4]/div/div/div/div/ul/li[1]/a').click()

        time.sleep(0.3)
        driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div[2]/div[2]/div/div[2]/div[6]').click()

        webdriver.ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(0.3)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        time.sleep(0.3)
        
        # 리뷰 20개 미만 처리
        min = len(soup.select('.area_card_outer._item_wrapper > li'))
        
        # 상위 리뷰 20개 추출
        if min >= 3:
            try:
                for cnt in range(0,3):
                    rev = soup.select('.area_card_outer._item_wrapper > li > .area_review_content > div')[cnt].select('.desc')[0].get_text()
                    rev_score = int(soup.select('.area_card_outer._item_wrapper > li .area_text_box')[cnt].get_text().split(')')[1])
                    new_row = pd.DataFrame({'영화명':[mname], '관람객 평점':[tot_score],
                           '남자 평점':[man_score], '여자 평점':[girl_score],
                           '10대 평점':[age10_score], '20대 평점':[age20_score],
                           '30대 평점':[age30_score], '40대 평점':[age40_score],
                           '50대 평점':[age50_score], '리뷰 평점':[rev_score], '리뷰':[rev],
                           '포스터' : [movie_img]})
                    data = pd.concat([data, new_row], axis=0)
                time.sleep(0.3)
                print(f'댓글 : {cnt+1}개')
            except:
                continue
        else:
            try:
                for cnt in range(0,min):
                    rev = soup.select('.area_card_outer._item_wrapper > li > .area_review_content > div')[cnt].select('.desc')[0].get_text()
                    rev_score = int(soup.select('.area_card_outer._item_wrapper > li .area_text_box')[cnt].get_text().split(')')[1])
                    new_row = pd.DataFrame({'영화명':[mname], '관람객 평점':[tot_score],
                           '남자 평점':[man_score], '여자 평점':[girl_score],
                           '10대 평점':[age10_score], '20대 평점':[age20_score],
                           '30대 평점':[age30_score], '40대 평점':[age40_score],
                           '50대 평점':[age50_score], '리뷰 평점':[rev_score], '리뷰':[rev],
                           '포스터' : [movie_img]})
                    data = pd.concat([data, new_row], axis=0)
                time.sleep(0.3)
                print(f'댓글 : {cnt+1}개')
            except:
                continue
    except:
        ec = ec + 1
        print(f'--------{mname} Error-------{ec}개')
        continue
        


In [ ]:
# data.to_csv('./reviewsample0511_s.csv', index=False)